In [1]:
from google.colab import drive
drive.mount('/content/drive')

# %%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.6/293.6 MB 6.5 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-68cg_1e1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-68cg_1e1
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936468 sha256=765456d8a2cca5118cbdd9e8ef318ad6c4d989fbed4566c0304082ab218d13bc
  Sto

In [2]:
!pip install bitsandbytes
!pip install unsloth_zoo
!pip install msgspec
!pip install blake3
!pip install gguf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 4.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.8.2 requires blake3, which is not installed.
vllm 0.8.2 requires compressed-tensors==0.9.2, which is not installed.
vllm 0.8.2 requires depyf==0.18.0, which is not installed.
vllm 0.8.2 requires fastapi[standard]>=0.115.0, which is not installed.
vllm 0.8.2 requires gguf==0.10.0, which is not installed.
vllm 0.8.2 requires lark==1.2.2, which is not installed.
vllm 0.8.2 requires llguidance<0.8.0,>=0.7.9; platform_machine == "x86_64" or platform_machine == "arm64" or platform_machine == "aarch64", which is not installed.
vllm 0.8.2 requires lm-format-enforcer<0.11,>=0.10.11, which is not installed.
vllm 0.8.2 requires mistral_common[opencv]>=1.5.4, which is not installed.
vllm 0.8.2 requires ninja, which is not installed.
vllm 0.

In [7]:
# 1. Import required libraries
import json
import torch
from unsloth import FastModel
from transformers import TextStreamer
from pathlib import Path
import time
import re

# 2. Define paths and settings
base_dir = "/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic"
# model_path =
model_path = "/content/drive/MyDrive/2025_ConvFinQA_SFT_Agentic/gemma-3-25_03_26_11AM-final_answer"
data_dir = Path(f"{base_dir}/data/processed_datasets")
dev_path = data_dir / "finqa_final_answer_dev.jsonl"

# Toggle for verbose output - set to True to see all examples
VERBOSE_OUTPUT = True

# Maximum number of examples to evaluate (set to None to evaluate all)
MAX_EXAMPLES: int | None = 100

# 3. Load the fine-tuned model and tokenizer
print("Loading fine-tuned model...")
model, tokenizer = FastModel.from_pretrained(
    model_name = model_path,
    max_seq_length = 4096,
    load_in_4bit = True,
)

# 4. Set up the tokenizer with Gemma 3 chat template
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
print("Model loaded successfully!")


Loading fine-tuned model...
==((====))==  Unsloth 2025.3.18: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Model loaded successfully!


In [8]:

# 5. Load all dev examples
print("Loading development examples...")
with open(dev_path, 'r') as f:
    dev_data = [json.loads(line) for line in f]
print(f"Loaded {len(dev_data)} dev examples")

def test_model_on_example(example: dict, model, tokenizer) -> dict:
    """Test the model on a single example using recommended Gemma 3 parameters"""
    # Create message format
    messages = [{
        "role": "user",
        "content": [{
            "type": "text",
            "text": example["prompt"]
        }]
    }]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt = True,
    )

    # Generate response using recommended Gemma 3 parameters
    start_time = time.time()
    outputs = model.generate(
        **tokenizer([text], return_tensors="pt").to("cuda"),
        max_new_tokens = 128,  # Shorter for final answers
        temperature = 0.7,     # Lower temperature for more focused answers
        top_p = 0.95,         # Gemma 3 recommended
        top_k = 64,           # Gemma 3 recommended
    )
    end_time = time.time()

    # Extract model output
    model_output = tokenizer.decode(outputs[0]).split("<start_of_turn>model\n")[1]
    if "<end_of_turn>" in model_output:
        model_output = model_output.split("<end_of_turn>")[0]

    generation_time = end_time - start_time

    return {
        "model_output": model_output.strip(),
        "expected_output": example["answer"],
        "generation_time": generation_time,
        "prompt": example["prompt"]
    }

def is_numeric_match(expected: str, generated: str) -> bool:
    """Check if numbers in the answers match, including handling rounding differences"""
    try:
        # Convert percentage strings to floats
        expected_num = float(expected.strip('%')) if '%' in expected else float(expected)
        generated_num = float(generated.strip('%')) if '%' in generated else float(generated)

        # If both are percentages or both are not percentages
        if ('%' in expected) == ('%' in generated):
            # For percentages, allow 0.2 percentage point difference
            if '%' in expected:
                return abs(expected_num - generated_num) <= 0.2
            # For non-percentages, allow 1% relative difference
            else:
                relative_diff = abs(expected_num - generated_num) / max(abs(expected_num), 0.0001)
                return relative_diff < 0.01
        return False
    except ValueError:
        return False

# Run evaluation
print("\n" + "="*80)
print("RUNNING FINAL ANSWER EVALUATION")
print("="*80)

if VERBOSE_OUTPUT:
    print("Verbose output enabled - showing all examples")
else:
    print("Verbose output disabled - showing summary only")

# Track metrics
all_results = []
total_time = 0
exact_match_count = 0
numeric_match_count = 0

# Run evaluation
dev_data_to_evaluate = dev_data[:MAX_EXAMPLES] if MAX_EXAMPLES is not None else dev_data
total_examples = len(dev_data_to_evaluate)
print(f"Starting evaluation on {total_examples} examples...")

# Create output file for all predictions
predictions_file = f"{base_dir}/evaluation_outputs/final_answer_predictions.jsonl"
with open(predictions_file, 'w') as pred_file:
    for i, example in enumerate(dev_data_to_evaluate):
        # Print progress updates
        if not VERBOSE_OUTPUT and ((i+1) % 25 == 0 or i+1 == total_examples):
            print(f"Progress: {i+1}/{total_examples} examples evaluated")

        # Run the model on this example
        result = test_model_on_example(example, model, tokenizer)

        # Clean up answers for comparison
        expected_answer = result["expected_output"].strip()
        model_answer = result["model_output"].strip()

        # Check for exact match
        exact_match = expected_answer.lower() == model_answer.lower()
        if exact_match:
            exact_match_count += 1
            match_status = "exact"
        else:
            # Check for numeric match with improved tolerance
            numeric_match = is_numeric_match(expected_answer, model_answer)
            if numeric_match:
                numeric_match_count += 1
                match_status = "numeric"
            else:
                match_status = "incorrect"

        # Save results
        result["exact_match"] = exact_match
        result["numeric_match"] = numeric_match
        result["match_status"] = match_status

        all_results.append(result)
        total_time += result["generation_time"]

        # Write individual result to JSONL file
        pred_file.write(json.dumps({
            "id": example.get("id", f"example_{i}"),
            "expected_answer": expected_answer,
            "model_answer": model_answer,
            "exact_match": exact_match,
            "numeric_match": numeric_match,
            "match_status": match_status,
        }) + "\n")

        # Print verbose output if enabled
        if VERBOSE_OUTPUT:
            print(f"\n{'-'*80}")
            print(f"Example {i+1}/{total_examples} [{match_status.upper()}]")
            print(f"Expected: {expected_answer}")
            print(f"Model: {model_answer}")
            print(f"Generation time: {result['generation_time']:.2f} seconds")

# Calculate and print evaluation metrics
avg_generation_time = total_time / total_examples
exact_match_accuracy = (exact_match_count / total_examples) * 100
numeric_match_accuracy = ((exact_match_count + numeric_match_count) / total_examples) * 100

print("\n" + "="*80)
print("EVALUATION RESULTS")
print("="*80)
print(f"Total examples evaluated: {total_examples}")
print(f"Average generation time: {avg_generation_time:.2f} seconds")
print(f"Exact match accuracy: {exact_match_accuracy:.2f}%")
print(f"Numeric match accuracy (includes exact + approximate): {numeric_match_accuracy:.2f}%")
print(f"Exact matches: {exact_match_count}/{total_examples}")
print(f"Numeric-only matches: {numeric_match_count}/{total_examples}")

# Save detailed results to JSON
results_file = f"{base_dir}/evaluation_outputs/final_answer_evaluation_results.json"
with open(results_file, 'w') as f:
    json.dump({
        "summary": {
            "total_examples": total_examples,
            "exact_match_accuracy": exact_match_accuracy,
            "numeric_match_accuracy": numeric_match_accuracy,
            "exact_match_count": exact_match_count,
            "numeric_match_count": numeric_match_count,
            "avg_generation_time": avg_generation_time,
        },
        "results": [{
            "expected_answer": r["expected_output"],
            "model_answer": r["model_output"],
            "match_status": r["match_status"],
            "generation_time": r["generation_time"]
        } for r in all_results]
    }, f, indent=2)

print(f"\nDetailed results saved to {results_file}")
print("\nEvaluation complete!")

Loading development examples...
Loaded 542 dev examples

RUNNING FINAL ANSWER EVALUATION
Verbose output enabled - showing all examples
Starting evaluation on 100 examples...

--------------------------------------------------------------------------------
Example 1/100 [INCORRECT]
Expected: 142.4%
Model: 134.8%
Generation time: 28.37 seconds

--------------------------------------------------------------------------------
Example 2/100 [INCORRECT]
Expected: -21.1%
Model: -16%
Generation time: 0.94 seconds

--------------------------------------------------------------------------------
Example 3/100 [INCORRECT]
Expected: 37.5%
Model: -21.4%
Generation time: 1.31 seconds

--------------------------------------------------------------------------------
Example 4/100 [INCORRECT]
Expected: 10.83%
Model: 11.1%
Generation time: 1.11 seconds

--------------------------------------------------------------------------------
Example 5/100 [INCORRECT]
Expected: -10%
Model: -8.8%
Generation time: 